In [2]:
import pandas as pd
import os
import missingno as msno
from sklearn.impute import KNNImputer
from tqdm import tqdm

In [ ]:
tracker = pd.read_csv('data/tracker.csv')

In [35]:
# combingin the
directory = 'data/train_data/'

# Initialize an empty dataframe
combined_df = pd.DataFrame()

# Loop through files in the directory

for filename in os.listdir(directory):
    if (filename.startswith('z')):  # Assuming the files are in CSV format
        file_path = os.path.join(directory, filename)
        # Read the file into a dataframe
        df = pd.read_csv(file_path)
        # Add a column with the file name
               
        df = df.rename(columns={'Unnamed: 0': 'timestamp'})
                
        # Concatenate the dataframe to the combined dataframe
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed')
               
        imputed_data = df.ffill().bfill()
        
        # Convert the result back to a DataFrame
        imputed_df = pd.DataFrame(imputed_data, columns=df.columns)
        imputed_df.set_index('timestamp', inplace=True)
        imputed_df = imputed_df.resample('1h').mean()
        
        imputed_df['File Name'] = filename
        
        imputed_df.to_csv(f'data/train_data_imputed/{filename}', index=True)               
        
        combined_df = pd.concat([combined_df, df], ignore_index=True)

combined_df = combined_df.rename(columns={'Unnamed: 0': 'timestamp'})

In [37]:
df_clean_test = pd.read_csv('data/train_data_imputed/z8jfojffxz.csv')


In [39]:
df_clean_test['date'] = pd.to_datetime(df_clean_test['timestamp']).dt.date

In [70]:
tracker = pd.read_csv('data/train_data/tracker.csv')

In [86]:
date_columns

['date']

In [85]:
# Convert columns with 'Date' in their name to date format
date_columns = [col for col in df_clean_test.columns if 'Date' in col or 'date' in col]
df_clean_test[date_columns] = df_clean_test[date_columns].apply(pd.to_datetime, format='%Y-%m-%d')


In [49]:
tracker_filtered = tracker[tracker['API'].isin(df_clean_test['File Name'].str[:-4])]

In [54]:
tracker_filtered.sort_values(by='Install #', inplace=True)

/tmp/ipykernel_162741/438238233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracker_filtered.sort_values(by='Install #', inplace=True)


In [71]:
tracker = tracker.sort_values(by=['API', 'Install #'])

0        3/3/2023 0:00
1      10/21/2023 0:00
3       1/15/2021 0:00
2        6/9/2021 0:00
4        6/2/2022 0:00
            ...       
200    12/31/2023 0:00
203     9/15/2020 0:00
202     7/11/2021 0:00
201      8/2/2021 0:00
204      1/9/2023 0:00
Name: Ins Date, Length: 205, dtype: object

In [87]:
date_cols = [col for col in tracker.columns if 'Date' in col or 'date' in col]
# Convert date columns to datetime, original format is 3/3/2023 0:00, target format is %Y-%m-%d 
for col in date_cols:
    tracker[col] = pd.to_datetime(tracker[col], format='%m/%d/%Y %H:%M')

In [ ]:
combined_df = pd.read_csv('data/train_data/combined_data.csv')

In [ ]:
msno.matrix(combined_df, figsize=(20, 10), fontsize=12)

In [ ]:
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'], format='mixed')
combined_df['day'] = combined_df['timestamp'].dt.day


In [88]:
tracker['Startup Date'].info()

<class 'pandas.core.series.Series'>
Index: 205 entries, 0 to 204
Series name: Startup Date
Non-Null Count  Dtype         
--------------  -----         
53 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 3.2 KB


In [ ]:
percentage = (nan_counts / len(combined_df)) * 100
#print(percentage)
#print(nan_counts)

In [ ]:
# preparing for downsampling
combined_df.set_index('timestamp', inplace=True)

In [ ]:
# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=2)

# Fit and transform the DataFrame
imputed_data = imputer.fit_transform()

# Convert the result back to a DataFrame
imputed_df = pd.DataFrame(imputed_data, columns=df.columns)



In [ ]:
combined_df.head()

In [ ]:
combined_df.resample('1D').mean().interpolate(method='linear', limit_direction='both', inplace=True)

In [ ]:
combined_df.head()